# Kaggle house prices with fastai v2
> Kaggling with fastai2

In [ ]:
#hide
from kaggle_house_prices.utils import *
from kaggle_house_prices.preprocessing import *
from kaggle_house_prices.modelling import *

## How to use